In [ ]:
import random
import time

# Directions for moving the blank space
DIRECTIONS = {
    (-1, 0): "Up",
    (1, 0): "Down",
    (0, -1): "Left",
    (0, 1): "Right"
}

# Opposite directions to prevent immediate reverses
OPPOSITE_DIRECTIONS = {
    "Up": "Down",
    "Down": "Up",
    "Left": "Right",
    "Right": "Left"
}


def is_solvable(state, n):
    """Check if a puzzle is solvable."""
    flat_state = [tile for row in state for tile in row if tile != 0]
    inversions = sum(
        flat_state[i] > flat_state[j] for i in range(len(flat_state)) for j in range(i + 1, len(flat_state))
    )
    return inversions % 2 == 0


def goal_state(n):
    """Return the goal state for the n-puzzle."""
    return [list(range(i * n, (i + 1) * n)) for i in range(n)]


def find_blank(state):
    """Find the position of the blank (zero) tile."""
    for i, row in enumerate(state):
        for j, tile in enumerate(row):
            if tile == 0:
                return i, j


def neighbors(state):
    """Return all valid neighboring states from the current state."""
    n = len(state)
    blank_i, blank_j = find_blank(state)
    possible_moves = []

    for di, dj in DIRECTIONS.keys():
        new_i, new_j = blank_i + di, blank_j + dj
        if 0 <= new_i < n and 0 <= new_j < n:
            new_state = [row[:] for row in state]
            new_state[blank_i][blank_j], new_state[new_i][new_j] = new_state[new_i][new_j], new_state[blank_i][blank_j]
            possible_moves.append((new_state, DIRECTIONS[(di, dj)]))

    return possible_moves


def scramble_puzzle(puzzle, n, max_moves):
    """Scramble the goal state with a limited number of random valid moves."""
    last_move = None
    for _ in range(max_moves):
        possible_moves = neighbors(puzzle)
        if last_move:
            # Filter out the move that would undo the last move
            possible_moves = [move for move in possible_moves if move[1] != OPPOSITE_DIRECTIONS[last_move]]
        if not possible_moves:
            break  # No valid moves available
        next_state, action = random.choice(possible_moves)
        puzzle = next_state
        last_move = action
    return puzzle


def generate_scrambled_puzzle(n, max_moves=10, completely_random=False):
    """Generate a puzzle for n x n grid with different strategies."""
    if completely_random:
        # Generate a completely random puzzle for n=3
        while True:
            tiles = list(range(n * n))
            random.shuffle(tiles)
            puzzle = [tiles[i:i + n] for i in range(0, len(tiles), n)]
            if is_solvable(puzzle, n):
                return puzzle
    else:
        # For n=4, 5, 6, scramble the goal state with limited random moves
        return scramble_puzzle(goal_state(n), n, max_moves)


def dfs_iterative(state, goal, depth_limit):
    """Iterative DFS without revisit checks but avoiding immediate reverses."""
    stack = [(state, [], 0, None)]  # Stack stores (state, path, depth, last_action)
    max_states_stored = 1

    while stack:
        current_state, path, depth, last_action = stack.pop()
        max_states_stored = max(max_states_stored, len(stack))

        if current_state == goal:
            return path, len(path), max_states_stored

        if depth < depth_limit:
            for neighbor, action in neighbors(current_state):
                # Avoid immediate reverse move
                if last_action and OPPOSITE_DIRECTIONS[last_action] == action:
                    continue
                stack.append((neighbor, path + [action], depth + 1, action))

    return None, 0, max_states_stored  # No solution found


def run_puzzle_solver_dfs(n_values=[3, 4, 5, 6], repetitions=10, max_moves=10, buffer=10):
    """
    Run the DFS solver for multiple puzzle sizes and repetitions.

    Args:
        n_values (list): List of puzzle sizes (n x n).
        repetitions (int): Number of runs per puzzle size.
        max_moves (int): Number of scrambling moves for n=4,5,6.
        buffer (int): Additional depth limit beyond scrambling moves.
    """
    for n in n_values:
        print(f"\nSolving {n ** 2 - 1}-puzzle (n = {n}):")
        solution_depths = []
        max_memory_usages = []

        for i in range(repetitions):
            if n == 3:
                # Generate completely random initial states for n=3
                initial = generate_scrambled_puzzle(n, completely_random=True)
                # Estimate solution path depth; BFS would find the shortest path, but we use DFS
                # So set depth_limit to a higher value
                depth_limit = 30
            else:
                # Generate initial states solvable in fewer steps for n=4, 5, 6
                initial = generate_scrambled_puzzle(n, max_moves=max_moves, completely_random=False)
                # Set depth_limit to max_moves + buffer
                depth_limit = max_moves + buffer

            goal = goal_state(n)

            # Print initial and goal state
            print(f"\nRun {i + 1}:")
            print("Initial State:")
            for row in initial:
                print(row)
            print("Goal State:")
            for row in goal:
                print(row)

            start_time = time.time()
            actions, solution_depth, max_states_stored = dfs_iterative(initial, goal, depth_limit=depth_limit)
            elapsed_time = time.time() - start_time

            if actions is not None:
                print(f"Solution depth: {solution_depth}")
                print(f"Max memory usage (states stored): {max_states_stored}")
                print(f"Solution sequence of actions: {', '.join(actions)}")
            else:
                print("No solution found.")

            # Collect statistics
            solution_depths.append(solution_depth if actions is not None else depth_limit)
            max_memory_usages.append(max_states_stored)

        # Report statistics
        print(f"\nStatistics for {n}-puzzle:")
        if solution_depths:
            print(
                f"Solution depth: Min = {min(solution_depths)}, Max = {max(solution_depths)}, Avg = {sum(solution_depths) / len(solution_depths):.1f}"
            )
            print(
                f"Max memory usage(states stored): Min = {min(max_memory_usages)}, Max = {max(max_memory_usages)}, Avg = {sum(max_memory_usages) / len(max_memory_usages):.1f}"
            )
        else:
            print("No solutions found.")


# Run the DFS solver for puzzles of size n = 3, 4, 5, 6 with 10 repetitions each
run_puzzle_solver_dfs()
